# Generate the final table of NDC to active ingredient mappings

2019-06-03

Here we consolidate the results and generate the final mapping of NDC to active ingredients.

This notebook ensures that we get a single, consistent set of active ingredients for each individual NDC.

In [1]:
import pandas as pd
from collections import defaultdict

## Read RXCUI to active ingredient RXCUI results

In [2]:
ingredients = pd.read_csv(
    "../../pipeline/ingredients/rxcui_ingredients/rxcui_ingredients_version_7.tsv",
    sep='\t'
)

In [3]:
ingredients.shape

(43556, 2)

In [4]:
ingredients.head()

,rxcui,active_ingredients
0,91349,5499
1,91792,7813
2,92582,30145
3,92583,30145
4,92584,30145


## Read NDC metadata

In [5]:
ndc = pd.read_csv("../../pipeline/merged_ndc_info.tsv", sep='\t')

In [6]:
ndc.shape

(246695, 20)

In [7]:
ndc.head(2)

,NDCPACKAGECODE,rxcui,suppress,PRODUCTNDC,PACKAGEDESCRIPTION,PRODUCTTYPENAME,PROPRIETARYNAME,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,0002-0800-01,540930,False,0002-0800,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,HUMAN OTC DRUG,Sterile Diluent,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,NDA,NDA018781,Eli Lilly and Company,WATER,1,mL/mL,NaN,NaN,N,20191231.0
1,0002-1200-30,1297712,False,0002-1200,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",HUMAN PRESCRIPTION DRUG,Amyvid,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,NDA,NDA202008,Eli Lilly and Company,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20191231.0


---

# Check that the results are consistent for NDCs with multiple RXCUIs

In [8]:
summary = (ndc
    [["NDCPACKAGECODE", "rxcui", "suppress"]]
    .merge(
        ingredients, how="inner", on="rxcui"
    )
    .drop_duplicates()
)

In [9]:
summary.shape

(246677, 4)

In [10]:
summary.head()

,NDCPACKAGECODE,rxcui,suppress,active_ingredients
0,0002-0800-01,540930,False,11295
1,0006-4309-00,540930,False,11295
2,0264-7385-50,540930,False,11295
3,0264-7385-60,540930,False,11295
4,0264-7850-00,540930,False,11295


### Add in number of RXCUIs per NDC

In [11]:
ncuis = (summary
    .groupby("NDCPACKAGECODE")
    ["rxcui"]
    .nunique()
    .to_frame("ncuis")
    .reset_index()
)

In [12]:
ncuis.shape

(244989, 2)

In [13]:
ncuis.head()

,NDCPACKAGECODE,ncuis
0,0002-0800-01,1
1,0002-1200-30,1
2,0002-1200-50,1
3,0002-1407-01,1
4,0002-1433-61,1


In [14]:
ncuis["ncuis"].value_counts()

1    243321
2      1648
3        20
Name: ncuis, dtype: int64

In [15]:
stats = (summary
    .merge(
        ncuis, how="inner", on="NDCPACKAGECODE"
    )
    .assign(
        good_ans = lambda df: df["active_ingredients"].map(
            lambda v: v != "-1"
        )
    )
)

In [16]:
stats.shape

(246677, 6)

In [17]:
stats.head()

,NDCPACKAGECODE,rxcui,suppress,active_ingredients,ncuis,good_ans
0,0002-0800-01,540930,False,11295,1,True
1,0006-4309-00,540930,False,11295,1,True
2,0264-7385-50,540930,False,11295,1,True
3,0264-7385-60,540930,False,11295,1,True
4,0264-7850-00,540930,False,11295,1,True


### Check that we get consistent results for NDCs with multiple RXCUIs

In [18]:
weird = []
for label, df in stats.query("ncuis > 1").groupby("NDCPACKAGECODE"):
    if df.query("good_ans")["active_ingredients"].nunique() > 1:
        weird.append(label)

In [19]:
len(weird)

5

In [20]:
(pd
    .Series(weird)
    .to_frame("NDCPACKAGECODE")
    .merge(stats, how="left", on="NDCPACKAGECODE")
    .sort_values(["NDCPACKAGECODE", "suppress"])
)

,NDCPACKAGECODE,rxcui,suppress,active_ingredients,ncuis,good_ans
0,52584-360-01,1668250,False,1908,2,True
1,52584-360-01,1867737,True,1901,2,True
2,52584-360-03,1668248,False,1908,2,True
3,52584-360-03,1867737,True,1901,2,True
4,68001-285-36,1803932,False,6313,2,True
5,68001-285-36,1720771,True,877015,2,True
6,68001-285-37,1803937,False,6313,2,True
7,68001-285-37,1720771,True,877015,2,True
8,68001-285-40,1803930,False,6313,2,True
9,68001-285-40,1720771,True,877015,2,True


After checking these examples by hand, I concluded that the suppressed values are incorrect.

---

# Harmonize results

There can be multiple RXCUIs for each NDC.
Some of the RXCUIs for the same NDC will not be in the semantic network, and therefore we won't find active ingredients for those RXCUIs.

However, this does not mean that we did not manage to find the active ingredients for that NDC through another RXCUI.

We will use the suppress status from RxNorm to determine which rows to keep as our final mapping.

### Note

There are some NDCs where all the RXCUIs are suppressed, but we managed to find an active ingredient for one of the RXCUIs anyways.

Example: 58789-005-01

In these instances we will take only the good answers.

In [21]:
def subset(df):
    if not df["suppress"].all():
        return df.query("~suppress")
    
    if df["good_ans"].any():
        return df.query("good_ans")
    
    return df


multi = (stats
    .query("ncuis > 1")
    .drop("ncuis", axis=1)

    .groupby("NDCPACKAGECODE")
    .apply(subset)
    .reset_index(drop=True)
)

In [22]:
multi.shape

(1690, 5)

In [23]:
multi.head()

,NDCPACKAGECODE,rxcui,suppress,active_ingredients,good_ans
0,0003-0315-05,2168097,False,10761,True
1,0003-0315-20,2168083,False,10761,True
2,0003-2188-11,1145932,False,614391,True
3,0003-2188-21,1145932,False,614391,True
4,0003-2188-50,1799230,False,614391,True


### Stats about NDCs with multiple RXCUIs

In [24]:
multi.groupby("NDCPACKAGECODE")["good_ans"].any().value_counts()

True     1648
False      20
Name: good_ans, dtype: int64

In [25]:
multi.groupby("NDCPACKAGECODE").size().value_counts()

1    1650
2      14
3       4
dtype: int64

### Check that there's only one answer for each NDC

In [26]:
multi.groupby("NDCPACKAGECODE")["active_ingredients"].nunique().value_counts()

1    1668
Name: active_ingredients, dtype: int64

In [27]:
multi.groupby("NDCPACKAGECODE")["good_ans"].nunique().value_counts()

1    1668
Name: good_ans, dtype: int64

### Rejoin with single RXCUI NDCs

In [28]:
res = (stats
    .query("ncuis == 1")
    .drop(["ncuis", "good_ans"], axis=1)

    .append(multi.drop("good_ans", axis=1))

    .sort_values(["NDCPACKAGECODE", "rxcui", "suppress", "active_ingredients"])
    .reset_index(drop=True)
)

In [29]:
res.shape

(245011, 4)

In [30]:
res.head()

,NDCPACKAGECODE,rxcui,suppress,active_ingredients
0,0002-0800-01,540930,False,11295
1,0002-1200-30,1297712,False,-1
2,0002-1200-50,1297712,False,-1
3,0002-1407-01,853004,False,35220
4,0002-1433-61,1551300,False,1551291


### Verify data behaves in the ways we expect it to

In [31]:
res.isnull().sum()

NDCPACKAGECODE        0
rxcui                 0
suppress              0
active_ingredients    0
dtype: int64

In [32]:
res["NDCPACKAGECODE"].nunique()

244989

In [33]:
res["rxcui"].nunique()

43231

In [34]:
res["suppress"].value_counts()

False    240291
True       4720
Name: suppress, dtype: int64

### Check that there's only one result for each NDC

In [35]:
(res
    [["NDCPACKAGECODE", "active_ingredients"]]
    .drop_duplicates()
    .groupby("NDCPACKAGECODE")
    .size()
    .value_counts()
)

1    244989
dtype: int64

---

## Save to file

In [36]:
res.to_csv("../../pipeline/ingredients/ndc_tables/ndc_to_rxcui_map_version_2.tsv", sep='\t', index=False)